In [ ]:
input_path = r'image.jpg'
modality = 'image' # 'image', 'video' , 'audio' , 'point','imu','fmri','depth','normal

<function torch.nn.init.normal_(tensor: torch.Tensor, mean: float = 0.0, std: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>

In [8]:
nn.Linear(16,16)

Linear(in_features=16, out_features=16, bias=True)

In [5]:
from fairscale.nn.model_parallel.layers import (
    ParallelEmbedding,
    RowParallelLinear,
    ColumnParallelLinear,
)
from fairscale.nn.model_parallel import initialize as fs_init

from torch import nn

ParallelEmbedding(
            1024, 2, init_method=nn.init.normal_,
        )

AssertionError: model parallel group is not initialized

In [6]:
import torchvision.transforms as transforms
from PIL import Image
from model.meta import MetaModel
import torch
from data.conversation_lib import conv_templates, SeparatorStyle


T_random_resized_crop = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=3,
                                 antialias=None),  # 3 is bicubic
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])])



model = MetaModel(llama_type='onellm', llama_config=r'config/llama2/7B.json',
                  tokenizer_path=r'config/llama2/tokenizer.model')

# load weights
checkpoint = torch.load(r'weights_dir/consolidated.00-of-01.pth', map_location='cpu')
msg = model.load_state_dict(checkpoint, strict=False)
print("load result:\n", msg)
model.cuda()
model.eval()
print(f"Model = {str(model)}")
# load data (IMAGE)
image = Image.open(input_path).convert('RGB')
inputs = T_random_resized_crop(image)
conv = conv_templates["v1"].copy()

# conv.append_message(conv.roles[0], user)
# conv.append_message(conv.roles[1], bot)
    
stream_response = model.stream_generate(
                'tell me more about this input', inputs,
                max_gen_len=1024, temperature=0.1, top_p=0.75,
                modal = 'image'
            )
conv_sep = (
                    conv.sep
                    if conv.sep_style == SeparatorStyle.SINGLE
                    else conv.sep2
                )
print(stream_response)

end_pos = stream_response["text"].find(conv_sep)
if end_pos != -1:
    stream_response["text"] = (
        stream_response['text'][:end_pos].rstrip() + "\n"
    )
    stream_response["end_of_content"] = True

# keep a few characters if not end_of_content to avoid sending
# part of conv_sep before all of it is generated.
if not stream_response["end_of_content"]:
    stream_response["text"] = (
        stream_response["text"][:-len(conv_sep)]
    )
    
    

ModuleNotFoundError: No module named 'flash_attn'